In [10]:
import pandas as pd
import os
import ast
import numpy as np
from Bio.SeqUtils.ProtParam import ProteinAnalysis
pd.options.display.max_colwidth = 100
pd.options.display.float_format = '{:.2f}'.format

In [11]:
all_results = []
for f in os.listdir('.'):
    if '.data' in f:
        with open(f, 'r') as g:
            all_results.append(ast.literal_eval(g.read()))

In [12]:
def standardize_to_uM(concentration, unit, sequence):
    concentration = concentration.replace(' ', '')
    try:
        concentration = float(concentration)
    except:
        return None
    if unit == 'uM' or unit == u'\xb5M' or unit == u'uM)':
        return concentration
    elif unit == 'ug/ml' or unit == u'\xb5g/ml' or unit == u'ug/ml)':
        try:
            molWt = ProteinAnalysis(sequence).molecular_weight()
        except ValueError:
            return None
        return concentration * 1000/molWt
    elif unit == 'nmol/g' or unit == 'pmol/mg':
        #1g, at density of 1g/mL, is 1mL, so nmol/g is nmol/mL = umol/L = uM yay!
        return concentration
    else:
        # print 'Unit not recognized: ' + unit
        return None

In [13]:
def convert_result_to_rows(sequence, result):
    rows = []
    if 'bacteria' not in result:
        return rows
    for bacterium, strain in result['bacteria']:
        
        rows.append({
            'bacterium': bacterium,
            'strain': strain,
            'sequence': sequence.upper(),
            'url_source': result['url_sources'][0],
            'value': standardize_to_uM(
                result['bacteria'][(bacterium, strain)]['value'],
                result['bacteria'][(bacterium, strain)]['unit'],
                sequence
            ),
            'modifications': result['modifications'] if 'modifications' in result else [],
            'unit': 'uM'
        })
        if rows[-1]['value']:
            rows[-1]['value'] = np.log10(rows[-1]['value'])
    return rows

In [14]:
rows = []
for result_set in all_results:
    for sequence in result_set:
        for row in convert_result_to_rows(sequence, result_set[sequence]):
            rows.append(row)

In [15]:
df = pd.DataFrame(rows)

In [16]:
def is_modified(modifications_list):
    return len(modifications_list) > 0

df['is_modified'] = df.modifications.apply(is_modified)

In [17]:
def has_non_cterminal_modification(modifications_list):
    return any(['C-Term' not in modification for modification in modifications_list])

df['has_non_cterminal_modification'] = df.modifications.apply(has_non_cterminal_modification)
#df['has_non_cterminal_modification'] = df.groupby(['sequence'])['has_non_cterminal_modification'].transform(max)

df['has_cterminal_modification'] = df.is_modified & ~df.has_non_cterminal_modification
#df['has_cterminal_modification'] = df.groupby(['sequence'])['has_cterminal_modification'].transform(max)

In [18]:
# Clean sequences
df.sequence = df.sequence.str.strip()
df = df.loc[df.sequence != '/']

In [19]:
# Exclude sequences with modifications

# Exclude rows from YADAMP and CAMP for having no modification data

#     Unless that sequence is in another DB

In [20]:
df = df.loc[df.has_non_cterminal_modification == False]

no_modification_data_sources = ['camp3', 'yadamp']

def datasource_has_modifications(cell):
    # Everything except CAMP and YADAMP has modification data
    return not any([s in cell for s in no_modification_data_sources])

df['_datasource_has_modifications'] = df['url_source'].apply(datasource_has_modifications)

sequences_containing_modifications = set(df.loc[df._datasource_has_modifications == True, 'sequence'])
def sequence_has_modification_data(cell):
    # If the sequence is labeled modifictationless in another database it's OK
    return cell in sequences_containing_modifications

df['_sequence_has_modifications'] = df['sequence'].apply(sequence_has_modification_data)

df['modification_verified'] = df['_sequence_has_modifications'] | df['_datasource_has_modifications']

df = df.loc[df.modification_verified == True]

In [399]:
character_dict = set([character for sequence in df.sequence for character in sequence])
max_sequence_length = int(df.sequence.str.len().describe(percentiles=[0.95])['95%'])
character_to_index = {
    character: i
    for i, character in enumerate(character_dict)
}

# max_sequence_length = 3


def sequence_to_vector(sequence, cterminal_amidation):
    default = np.zeros([max_sequence_length, len(character_to_index) + 1])
    for i, character in enumerate(sequence[:max_sequence_length]):
        default[i][character_to_index[character]] = 1
        default[i][-1] = cterminal_amidation
    return default


def row_to_vector(row):
    sequence = row['sequence']
    cterminal_amidation = row['has_cterminal_modification']
    default = np.zeros([max_sequence_length, len(character_to_index) + 1])

    for i, character in enumerate(sequence[:max_sequence_length]):
        default[i][character_to_index[character]] = 1
        default[i][-1] = cterminal_amidation

    return default

In [422]:
# ecoli_df = df.loc[(df.bacterium.str.contains('E. coli')) | (df.bacterium.str.contains('P. aeruginosa'))].groupby(['sequence', 'bacterium'])
ecoli_df = df.loc[(df.bacterium.str.contains('E. coli'))].groupby(['sequence', 'bacterium'])

ecoli_df = ecoli_df.mean().reset_index().dropna()

In [423]:
cterminal_amidation = np.array(ecoli_df.has_cterminal_modification)

In [424]:
# vectors = ecoli_df.head(2).apply(sequence_to_vector, axis=1)
vectors = []
for row in ecoli_df.iterrows():
    vectors.append(row_to_vector(row[1]))

In [429]:
vectors = np.array(vectors)
print(vectors.shape)

(4508, 46, 24)


In [426]:
labels = np.array(ecoli_df.value)

In [430]:
average = np.mean(labels)
squared_errors = sum([(label - average) ** 2 for label in labels])
baseline_error = squared_errors/len(labels)
print(baseline_error)

0.60182344038256


In [431]:
import keras
from keras.layers import Dense, Dropout, LSTM, Conv2D, Conv1D, MaxPooling1D, MaxPooling2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [411]:
def baseline_model():
    model = keras.models.Sequential()
    model.add(LSTM(
        128,
        input_shape=(max_sequence_length, len(character_to_index) + 1),
    ))
    """model.add(Dense(
        max_sequence_length * len(character_to_index),
        input_dim = max_sequence_length * len(character_to_index),
        kernel_initializer='normal',
        activation='relu'
    ))"""
    #model.add(Dropout(0.5))
    model.add(Dense(50, kernel_initializer='normal'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [435]:
def conv_model():
    model = keras.models.Sequential()
    model.add(Conv1D(
        64,
        kernel_size = 5,
        strides = 1,
        activation = 'relu',
        input_shape = (max_sequence_length, len(character_to_index) + 1)
    ))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    #model.add(Dropout(0.5))
    model.add(Conv1D(64, 5, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    #model.add(Dense(100, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [412]:
model = baseline_model()

In [413]:
cutoff = int(0.9 * len(labels))
train_x = vectors[:cutoff]
train_y = labels[:cutoff]
test_x = vectors[cutoff:]
test_y = labels[cutoff:]

In [414]:
model.fit(train_x, train_y, batch_size=40, epochs=100)

Epoch 1/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.6298
Epoch 2/100
6300/6300 [==============================] - 6s 977us/step - loss: 0.5520
Epoch 3/100
6300/6300 [==============================] - 6s 1ms/step - loss: 0.4722
Epoch 4/100
6300/6300 [==============================] - 6s 968us/step - loss: 0.4358
Epoch 5/100
6300/6300 [==============================] - 6s 1ms/step - loss: 0.4293
Epoch 6/100
6300/6300 [==============================] - 6s 1ms/step - loss: 0.4148
Epoch 7/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.4032
Epoch 8/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.4006
Epoch 9/100
6300/6300 [==============================] - 6s 1ms/step - loss: 0.3978
Epoch 10/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.3864
Epoch 11/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.3959
Epoch 12/100
6300/6300 [==============================] - 7s 1ms/step 

6300/6300 [==============================] - 8s 1ms/step - loss: 0.0886
Epoch 97/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.0919
Epoch 98/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.0925
Epoch 99/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.0882
Epoch 100/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.0871


In [348]:
model.fit(train_x, train_y, batch_size=40, epochs=100)

Epoch 1/100
6300/6300 [==============================] - 9s 1ms/step - loss: 0.6312
Epoch 2/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.5538
Epoch 3/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.4889
Epoch 4/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.4335
Epoch 5/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.4308
Epoch 6/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.4298
Epoch 7/100
6300/6300 [==============================] - 6s 1ms/step - loss: 0.4111
Epoch 8/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.4157
Epoch 9/100
6300/6300 [==============================] - 6s 989us/step - loss: 0.4039
Epoch 10/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.3983
Epoch 11/100
6300/6300 [==============================] - 7s 1ms/step - loss: 0.3947
Epoch 12/100
6300/6300 [==============================] - 7s 1ms/step - 

In [420]:
convmodel = conv_model()
convmodel.fit(train_x, train_y, batch_size=40, epochs=200)

Epoch 1/200
6300/6300 [==============================] - 1s 214us/step - loss: 0.4987
Epoch 2/200
6300/6300 [==============================] - 1s 135us/step - loss: 0.3770
Epoch 3/200
6300/6300 [==============================] - 1s 135us/step - loss: 0.3315
Epoch 4/200
6300/6300 [==============================] - 1s 138us/step - loss: 0.2954
Epoch 5/200
6300/6300 [==============================] - 1s 134us/step - loss: 0.2736
Epoch 6/200
6300/6300 [==============================] - 1s 135us/step - loss: 0.2539
Epoch 7/200
6300/6300 [==============================] - 1s 142us/step - loss: 0.2318
Epoch 8/200
6300/6300 [==============================] - 1s 144us/step - loss: 0.2281
Epoch 9/200
6300/6300 [==============================] - 1s 140us/step - loss: 0.2154
Epoch 10/200
6300/6300 [==============================] - 1s 140us/step - loss: 0.2007
Epoch 11/200
6300/6300 [==============================] - 1s 135us/step - loss: 0.1918
Epoch 12/200
6300/6300 [============================

6300/6300 [==============================] - 1s 147us/step - loss: 0.0925
Epoch 96/200
6300/6300 [==============================] - 1s 132us/step - loss: 0.0885
Epoch 97/200
6300/6300 [==============================] - 1s 154us/step - loss: 0.0935
Epoch 98/200
6300/6300 [==============================] - 1s 134us/step - loss: 0.0909
Epoch 99/200
6300/6300 [==============================] - 1s 134us/step - loss: 0.0897
Epoch 100/200
6300/6300 [==============================] - 1s 140us/step - loss: 0.0890
Epoch 101/200
6300/6300 [==============================] - 1s 136us/step - loss: 0.0922
Epoch 102/200
6300/6300 [==============================] - 1s 141us/step - loss: 0.0924
Epoch 103/200
6300/6300 [==============================] - 1s 152us/step - loss: 0.0888
Epoch 104/200
6300/6300 [==============================] - 1s 133us/step - loss: 0.0890
Epoch 105/200
6300/6300 [==============================] - 1s 134us/step - loss: 0.0889
Epoch 106/200
6300/6300 [=========================

6300/6300 [==============================] - 1s 132us/step - loss: 0.0769
Epoch 189/200
6300/6300 [==============================] - 1s 134us/step - loss: 0.0764
Epoch 190/200
6300/6300 [==============================] - 1s 134us/step - loss: 0.0739
Epoch 191/200
6300/6300 [==============================] - 1s 133us/step - loss: 0.0786
Epoch 192/200
6300/6300 [==============================] - 1s 135us/step - loss: 0.0752
Epoch 193/200
6300/6300 [==============================] - 1s 137us/step - loss: 0.0761
Epoch 194/200
6300/6300 [==============================] - 1s 138us/step - loss: 0.0765
Epoch 195/200
6300/6300 [==============================] - 1s 141us/step - loss: 0.0758
Epoch 196/200
6300/6300 [==============================] - 1s 147us/step - loss: 0.0756
Epoch 197/200
6300/6300 [==============================] - 1s 133us/step - loss: 0.0764
Epoch 198/200
6300/6300 [==============================] - 1s 133us/step - loss: 0.0747
Epoch 199/200
6300/6300 [=====================

In [433]:
convmodel = conv_model()
convmodel.fit(train_x, train_y, batch_size=40, epochs=200)

Epoch 1/200
6300/6300 [==============================] - 1s 210us/step - loss: 0.5337
Epoch 2/200
6300/6300 [==============================] - 1s 131us/step - loss: 0.3819
Epoch 3/200
6300/6300 [==============================] - 1s 132us/step - loss: 0.3358
Epoch 4/200
6300/6300 [==============================] - 1s 133us/step - loss: 0.3024
Epoch 5/200
6300/6300 [==============================] - 1s 133us/step - loss: 0.2780
Epoch 6/200
6300/6300 [==============================] - 1s 137us/step - loss: 0.2531
Epoch 7/200
6300/6300 [==============================] - 1s 132us/step - loss: 0.2410
Epoch 8/200
6300/6300 [==============================] - 1s 134us/step - loss: 0.2203
Epoch 9/200
6300/6300 [==============================] - 1s 134us/step - loss: 0.2113
Epoch 10/200
6300/6300 [==============================] - 1s 141us/step - loss: 0.1999
Epoch 11/200
6300/6300 [==============================] - 1s 149us/step - loss: 0.1988
Epoch 12/200
6300/6300 [============================

6300/6300 [==============================] - 1s 133us/step - loss: 0.0939
Epoch 96/200
6300/6300 [==============================] - 1s 134us/step - loss: 0.0940
Epoch 97/200
6300/6300 [==============================] - 1s 132us/step - loss: 0.0916
Epoch 98/200
6300/6300 [==============================] - 1s 131us/step - loss: 0.0896
Epoch 99/200
6300/6300 [==============================] - 1s 132us/step - loss: 0.0903
Epoch 100/200
6300/6300 [==============================] - 1s 132us/step - loss: 0.0888
Epoch 101/200
6300/6300 [==============================] - 1s 179us/step - loss: 0.0914
Epoch 102/200
6300/6300 [==============================] - 1s 150us/step - loss: 0.0900
Epoch 103/200
6300/6300 [==============================] - 1s 133us/step - loss: 0.0913
Epoch 104/200
6300/6300 [==============================] - 1s 133us/step - loss: 0.0892
Epoch 105/200
6300/6300 [==============================] - 1s 133us/step - loss: 0.0885
Epoch 106/200
6300/6300 [=========================

6300/6300 [==============================] - 1s 133us/step - loss: 0.0776
Epoch 189/200
6300/6300 [==============================] - 1s 140us/step - loss: 0.0785
Epoch 190/200
6300/6300 [==============================] - 1s 157us/step - loss: 0.0783
Epoch 191/200
6300/6300 [==============================] - 1s 133us/step - loss: 0.0791
Epoch 192/200
6300/6300 [==============================] - 1s 133us/step - loss: 0.0775
Epoch 193/200
6300/6300 [==============================] - 1s 131us/step - loss: 0.0765
Epoch 194/200
6300/6300 [==============================] - 1s 134us/step - loss: 0.0770
Epoch 195/200
6300/6300 [==============================] - 1s 138us/step - loss: 0.0753
Epoch 196/200
6300/6300 [==============================] - 1s 154us/step - loss: 0.0780
Epoch 197/200
6300/6300 [==============================] - 1s 138us/step - loss: 0.0757
Epoch 198/200
6300/6300 [==============================] - 1s 133us/step - loss: 0.0772
Epoch 199/200
6300/6300 [=====================

In [415]:
model.evaluate(test_x, test_y)

701/701 [==============================] - 0s 406us/step


0.522078353991352

In [434]:
convmodel.evaluate(test_x, test_y)

701/701 [==============================] - 0s 238us/step


0.4417123108379511

In [ ]:
train_x = 

In [37]:
vectors = np.array([list(vector) for vector in vectors]).reshape([-1, max_sequence_length, len(character_to_index)])

In [30]:
cterminal_amidation.shape

(7001,)

In [45]:
vectors[:, np.newaxis, :][0]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [9]:
df

,bacterium,modifications,sequence,strain,unit,url_source,value
0,B. subtilis,[disulfide],GLPRKILCAIAKKKGKCKGPLKLVCKC,None,uM,http://aps.unmc.edu/AP/database/query_output.php?ID=1958,-0.40
1,S. agalactiae,[disulfide],GFGCPGDAYQCSEHCRALGGGRTGGYCAGPWYLGHPTCTCSF,None,uM,http://aps.unmc.edu/AP/database/query_output.php?ID=2119,-1.86
2,S. aureus,[disulfide],GFGCPGDAYQCSEHCRALGGGRTGGYCAGPWYLGHPTCTCSF,ATCC29213,uM,http://aps.unmc.edu/AP/database/query_output.php?ID=2119,0.57
3,S. pneumonia,[disulfide],GFGCPGDAYQCSEHCRALGGGRTGGYCAGPWYLGHPTCTCSF,ATCC49619,uM,http://aps.unmc.edu/AP/database/query_output.php?ID=2119,-1.24
4,S. epidermidis,[disulfide],GFGCPGDAYQCSEHCRALGGGRTGGYCAGPWYLGHPTCTCSF,ATCC12228,uM,http://aps.unmc.edu/AP/database/query_output.php?ID=2119,0.57
5,E. faecium,[disulfide],GFGCPGDAYQCSEHCRALGGGRTGGYCAGPWYLGHPTCTCSF,ATCC49624,uM,http://aps.unmc.edu/AP/database/query_output.php?ID=2119,0.57
6,S. pyogenes,[disulfide],GFGCPGDAYQCSEHCRALGGGRTGGYCAGPWYLGHPTCTCSF,None,uM,http://aps.unmc.edu/AP/database/query_output.php?ID=2119,-1.86
7,S. aureus,[],LVKDNPLDISPKQVQALCTDLVIRCMCCC,None,uM,http://aps.unmc.edu/AP/database/query_output.php?ID=2341,0.49
8,C. michiganensis,[],LVKDNPLDISPKQVQALCTDLVIRCMCCC,None,uM,http://aps.unmc.edu/AP/database/query_output.php?ID=2341,0.49
9,R. solani,[],LVKDNPLDISPKQVQALCTDLVIRCMCCC,None,uM,http://aps.unmc.edu/AP/database/query_output.php?ID=2341,1.40
